<a href='https://github.com/SeWonKwon' ><div> <img src ='https://slid-capture.s3.ap-northeast-2.amazonaws.com/public/image_upload/6556674324ed41a289a354258718280d/964e5a8b-75ad-41fc-ae75-0ca66d06fbc7.png' align='left' /> </div></a>


1. 네이버 자료

2. 네이버 Place 코드
    * 기준: 서울시 동별, 업종별
    
    
3. 가게정보, 리뷰 정도 겟



# 필요함수 및 라이브러리

In [1]:
import time
import pandas as pd
import numpy as np
import json

from selenium import webdriver
from tqdm import tqdm_notebook
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import ElementClickInterceptedException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

import warnings
warnings.filterwarnings(action = 'ignore')


## 파일 관련

In [2]:
def get_today():
    import datetime
    now = datetime.datetime.now()
    
    nowDate = now.strftime('%Y-%m-%d')
#     print(nowDate)      # 2015-04-19
 
    nowTime = now.strftime('%H:%M:%S')
#     print(nowTime)      # 12:11:32

    nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')
    return nowDate

In [3]:
import os
def check_file(file_path, file_name):
    return os.path.isfile(file_path+file_name)

In [4]:
def save_df(df, path, file_name):
    df.to_csv(path+file_name, index=False)
    print(f'{path+file_name} 저장 완료')

In [5]:
def read_csv_코드정보(code_path , code_file):
    df = pd.read_csv(code_path + code_file )
    return df

In [6]:
'''음식점:DINING, 카페: CAFE, 쇼핑: SHOPPING, 숙박: ACCOMMODATION, 병원의료: HOSPITAL, 은행: BANK,
   주요소:OIL, 마트슈퍼: MART, 편의점: STORE, 생활편의: CONVENIENCE, 명소: SIGHTS, 체육시설: SPORT, 
   영화공연:CINEMA, 관공서:GOVERNMENT'''

'음식점:DINING, 카페: CAFE, 쇼핑: SHOPPING, 숙박: ACCOMMODATION, 병원의료: HOSPITAL, 은행: BANK,\n   주요소:OIL, 마트슈퍼: MART, 편의점: STORE, 생활편의: CONVENIENCE, 명소: SIGHTS, 체육시설: SPORT, \n   영화공연:CINEMA, 관공서:GOVERNMENT'

###### 서울시 동 load 관련

In [7]:
import pickle
import collections

def get_seoul_dong():
    with open('data/서울시_구별_동정리.pickle','rb') as fw:
        dong_dict = pickle.load( fw)

    dong_key_list = list(dong_dict.keys())
    dong_key_list.sort()
    # dong_key_list

    for 구이름 in dong_key_list:
        dong_dict[구이름] = sorted(dong_dict[구이름])
    
    return dong_dict

In [8]:
dong_dict = get_seoul_dong()

## 웹 조정 관련

In [9]:
def is_clickable(wd):
    return wd.is_enabled() and wd.is_displayed()

In [10]:
def scroll_down(wd,SCROLL_PAUSE_SEC = 1):

    # 스크롤 높이 가져옴
    last_height = wd.execute_script("return document.body.scrollHeight")
#     print(last_height)
    while True:
        # 끝까지 스크롤 다운
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 1초 대기
        time.sleep(SCROLL_PAUSE_SEC)

        # 스크롤 다운 후 스크롤 높이 다시 가져옴
        new_height = wd.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

## 크롤링 에러 방지 관련

In [11]:
def try_none_text_by_class_name(wd,class_name, col='col'):
    try:
        return wd.find_element_by_class_name(class_name).text
    except NoSuchElementException:
        print(col, end=' ')
        return np.NaN
        

In [12]:
def strong_try_none_text_by_class_name(wd,class_name, col='col'):
    try:
        temps = wd.find_elements_by_class_name(class_name)
        for temp in temps:
            if temp.text.startswith('편의'):
                return temp.text.split('\n')[1]
        else:
            return np.NaN
    except NoSuchElementException:
#         print(col, end='')
        return np.NaN
        

In [13]:
def review_dict(s):
    review_dict = {}
#     print(s.split('평균 별점'))
    s = s.split('평균 별점')
    review_dict['평균_별점'] = s[-1].strip()
    s = s[0].split('사진')
    if len(s) >= 2:
        review_dict['사진'] = int(s[-1])
        review_dict['리뷰'] = s[0].split()[-1]
    else:
        review_dict['사진'] = np.NaN
        review_dict['리뷰'] = s[0].split()[-1]
# print(s)
    return review_dict

## get_store_info from naver.place by place_code

In [14]:
def get_store_info(wd, code_no):
    name = wd.find_element_by_class_name('_3XamX').text
#     print(name)
    r_type = try_none_text_by_class_name(wd,'_3ocDE','ty')
#     print(r_type)
    address = try_none_text_by_class_name(wd,'_2yqUQ','ad')
#     print(address)
    tel = try_none_text_by_class_name(wd,'_3ZA0S','tel') 
#     print(tel)
    utility, detail = np.NaN, np.NaN
    for i in wd.find_elements_by_class_name('_1M_Iz'):
#         print(i.text)
        if '편의' in i.text :
            utility = i.text.replace('편의\n','')
        elif '설명' in i.text :
            detail = i.text.replace('설명\n', '')
    
#     print(code_no, utility, detail)
    
    nums_list = wd.find_elements_by_class_name('_1Y6hi')
    try:
        score = nums_list[0].find_element_by_tag_name('em').text
    except:
        score = np.NaN
    try:
        review_num = nums_list[1].find_element_by_tag_name('em').text
    except:
        review_num = np.NaN
        
    try:
        blog_nums = nums_list[2].find_element_by_tag_name('em').text
    except:
        blog_nums = np.NaN
#     print(score, review_num, blog_nums)
    collected_date = get_today()

    # 데이터 렙 펼치기
    for i in wd.find_elements_by_class_name('_3iTUo'):
        if i.text=='더보기':
            i.send_keys('\n')
        else:
            pass
            
    try:
        datalab_1 = {}
        datalab_list = wd.find_elements_by_class_name('_3Ryhx')
        for datalab in datalab_list:
            datalab_1[datalab.find_element_by_class_name('_3hvd9').text] = [x.text for x in datalab.find_elements_by_class_name('_2irYJ') if x.text != '']
    except:
        datalab_1 = np.NaN
        print('d_1', end='')
#     print(name, datalab_1)

    try:
        datalab_g = {}
        datalab_block = wd.find_element_by_class_name('_250cA')
        datalab_age_list = datalab_block.find_elements_by_class_name('_1Nyv0')
        for age_info in datalab_age_list:
            age = age_info.find_element_by_class_name('iO1iA').text
            preference = age_info.find_element_by_class_name('place_blind').text.replace('%','')
            datalab_g[age] = preference
            
        gender_preferance = wd.find_element_by_class_name('c3-chart-arc.c3-target.c3-target-female').text.replace('%',"")
        datalab_g['female_pref'] = gender_preferance
    except:
        datalab_g = np.NaN
        print('d_g', end="")
#     print(datalab_1, datalab_g)

    return name, code_no, r_type, address,tel, score, utility, detail, datalab_1, datalab_g, review_num, blog_nums, collected_date

In [15]:
code_path = 'data/코드/'
구이름 = '강남구'
업종 = 'DINING'
code_file = '가게코드_' + 구이름 + '_'+ 업종 + '.csv'

In [16]:
code_df = read_csv_코드정보(code_path, code_file)

In [17]:
code_df.shape

(6863, 2)

In [18]:
def naver_map_store_scrapping( 구이름, 업종, slow= 1):
    start_time = time.time()

    options = webdriver.ChromeOptions()

    options.add_argument("--start-maximized")
    wd = webdriver.Chrome(ChromeDriverManager().install(), options= options)  # chromedriver 열기
    wd.implicitly_wait(1*slow)
    
    code_path = 'data/코드/'
    review_path = 'data/리뷰/'
    store_path = 'data/가게/'

    code_file = '가게코드_' + 구이름 + '_'+ 업종 + '.csv'
    review_file = '리뷰정보_' + 구이름 + '_'+ 업종 + '.csv'
    store_file = '가게정보_' + 구이름 + '_'+ 업종 + '.csv'
    
    store_idx = 0
    store_df = pd.DataFrame(columns=("name", "code_no", "r_type","address","tel",
                                     "score","utility","detail","datalab_1", "datalab_g", "review_num",
                                     "blog_num", "collected_date"))
    
    if check_file(store_path, store_file):
        store_df = pd.read_csv(store_path+store_file)
        store_idx = store_df.shape[0]
    done_list = list(store_df['code_no'].apply(lambda x: str(x)).values)
#     print(done_list)
    code_df = pd.read_csv(code_path+code_file, dtype='str')
    code_list = code_df['store_code'].to_list()
    
    print(f'{구이름}  {업종} 업종 의 가게 정보 스크래핑을 시작합니다.')

    print(f'{code_df.shape[0]}개의 가게코드가 있습니다.')
    
    print(f'{store_idx}가 이전에 완료 되었었습니다. 이어서 진행합니다.')

    for code_no_ in tqdm_notebook(code_list):
        code_no = str(code_no_)
        if code_no in done_list:
#             print(code_no, '중복중복!!')
            continue
            
        place_url = 'https://pcmap.place.naver.com/restaurant/'+str(code_no)

        wd.get(place_url)
        time.sleep(1*slow)
        scroll_down(wd,0.5)
        cnt_idx = 0

        try:
#             input('ddd')
            wd.implicitly_wait(0.001*slow)
            store_df.loc[store_idx] = get_store_info(wd,code_no)
            wd.implicitly_wait(1*slow)
            print(store_idx, end='#')
            store_idx += 1
            time.sleep(5)
            
        except:
            print(store_idx, end='!!')
            pass
        
        ##################### cooling down
        if store_idx%100 == 0 :
            wd.close()
            wd = webdriver.Chrome(ChromeDriverManager().install(), options= options)  # chromedriver 열기
            wd.implicitly_wait(1*slow)
            time.sleep(1*slow)
            
        ##################### save just in case:
        if store_idx%30 == 0 :
            store_df.to_csv(store_path+store_file, index=False)
            print(구이름, 업종, f'가게 {store_idx}개 중간저장')
            print(f'{time.time()-start_time} 소요')
#         ##################### Test Test Test
#         if store_idx%10==0:
#             store_df.to_csv(store_path+store_file, index=False)
#             print(구이름,업종, f'가게 {store_idx}개 완료' )
#             print(f'{time.time()-start_time} 소요')
#             wd.close()
#             break
    else:
        store_df.to_csv(store_path+store_file, index=False)
        print(구이름, 업종, f'가게 {store_idx}개 완료' )
        print(f'{time.time()-start_time} 소요')
        wd.close()
        
#     return store_df

# 셋팅

In [19]:
# code_path = 'data/코드/'
# review_path = 'data/리뷰/'
# store_path = 'data/가게/'

# code_file = '가게코드_' + 구이름 + '_' + 동이름+ '_'+ 업종 + '.csv'
# review_file = '리뷰정보_' + 구이름 + '_' + 동이름+ '_'+ 업종 + '.csv'
# store_file = '가게정보_' + 구이름 + '_' + 동이름+ '_'+ 업종 + '.csv'

In [20]:
code_path = 'data/코드/'
store_path = 'data/가게/'
# code_file = '가게코드_' + 구이름 + '_'+ 업종 + '.csv'
# store_file = '가게정보_' + 구이름 +  '_'+ 업종 + '.csv'

In [21]:
detail= True
slow=1

#  코드 수집 및 저장

In [ ]:
업종= 'DINING'
idx = 0
while True:
    try:
        for 구이름 in sorted(dong_dict.keys()):
            code_file = '가게코드_' + 구이름 + '_'+ 업종 + '.csv'
            if check_file(code_path, code_file):
                store_file = '가게정보_' + 구이름 +  '_'+ 업종 + '.csv'
                naver_map_store_scrapping( 구이름, 업종, slow= 1)
            else:
                print(f'{구이름} {업종}이 없습니다. ')
                
    except:
        time.sleep(idx*idx)
        print(f'{idx} 실패')
        idx+=1
        
    if idx >50:
        print(f'{idx} 실패')
        break

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 94.0.4606
[WDM] - Get LATEST driver version for 94.0.4606
[WDM] - Driver [C:\Users\N\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


강남구  DINING 업종 의 가게 정보 스크래핑을 시작합니다.
6863개의 가게코드가 있습니다.
1260가 이전에 완료 되었었습니다. 이어서 진행합니다.


1260!!강남구 DINING 가게 1260개 중간저장
3.6730825901031494 소요
d_g1260#d_g1261#d_g1262#1263#d_g1264#d_g1265#d_g1266#d_g1267#d_g1268#d_g1269#1270#d_g1271#d_g1272#d_g1273#1274#d_g1275#d_g1276#tel d_g1277#1278#1279#1280#1281#1282#1283#1284#1285#1286#1287#1288#d_g1289#강남구 DINING 가게 1290개 중간저장
251.24064779281616 소요
1290#1291#1292#1293#d_g1294#d_g1295#d_g1296#1297#d_g1298#d_g1299#

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 94.0.4606
[WDM] - Get LATEST driver version for 94.0.4606
[WDM] - Driver [C:\Users\N\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


d_g1300#d_g1301#d_g1302#d_g1303#d_g1304#d_g1305#d_g1306#d_g1307#d_g1308#1309#d_g1310#d_g1311#1312#d_g1313#d_g1314#1315#d_g1316#d_g1317#1318#d_g1319#강남구 DINING 가게 1320개 중간저장
501.35474252700806 소요
1320#1321#1322#d_g1323#d_g1324#d_g1325#tel d_g1326#d_g1327#1328#d_g1329#1330#d_g1331#d_g1332#tel d_g1333#d_g1334#d_g1335#1336#tel d_g1337#1338#d_g1339#1340#d_g1341#d_g1342#1343#d_g1344#1345#d_g1346#1347#1348#d_g1349#강남구 DINING 가게 1350개 중간저장
737.1041333675385 소요
d_g1350#d_g1351#1352#1353#1354#d_g1355#1356#1357#d_g1358#d_g1359#d_g1360#d_g1361#1362#1363#d_g1364#1365#d_g1366#d_g1367#d_g1368#d_g1369#d_g1370#d_g1371#1372#1373#d_g1374#d_g1375#d_g1376#1377#d_g1378#d_g1379#강남구 DINING 가게 1380개 중간저장
972.2319996356964 소요
d_g1380#d_g1381#d_g1382#d_g1383#1384#d_g1385#d_g1386#d_g1387#d_g1388#d_g1389#d_g1390#d_g1391#d_g1392#d_g1393#1394#d_g1395#1396#1397#d_g1398#d_g1399#

...

강남구 DINING 가게 4800개 중간저장
29286.728705883026 소요
d_g4800#d_g4801#d_g4802#tel d_g4803#d_g4804#d_g4805#d_g4806#d_g4807#d_g4808#d_g4809#d_g4810#d_g4811#d_g4812#d_g4813#d_g4814#d_g4815#d_g4816#d_g4817#d_g4818#d_g4819#d_g4820#tel d_g4821#tel d_g4822#d_g4823#d_g4824#d_g4825#d_g4826#d_g4827#d_g4828#d_g4829#강남구 DINING 가게 4830개 중간저장
29518.953102588654 소요
d_g4830#4831#4832#4833#4834#4835#4836#4837#4838#4839#4840#4841#4842#d_g4843#4844#d_g4845#d_g4846#4847#4848#4849#d_g4850#4851#4852#4853#4854#4855#4856#d_g4857#4858#4859#강남구 DINING 가게 4860개 중간저장
29771.5138528347 소요
4860#d_g4861#d_g4862#4863#4864#d_g4865#d_g4866#4867#d_g4868#4869#d_g4870#d_g4871#d_g4872#4873#d_g4874#d_g4875#d_g4876#d_g4877#4878#d_g4879#4880#d_g4881#d_g4882#d_g4883#4884#d_g4885#d_g4886#d_g4887#d_g4888#4889#강남구 DINING 가게 4890개 중간저장
30019.00657582283 소요
4890#d_g4891#d_g4892#4893#d_g4894#d_g4895#d_g4896#d_g4897#d_g4898#d_g4899#

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 94.0.4606
[WDM] - Get LATEST driver version for 94.0.4606
[WDM] - Driver [C:\Users\N\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


d_g4900#d_g4901#d_g4902#d_g4903#d_g4904#d_g4905#tel d_g4906#d_g4907#d_g4908#d_g4909#d_g4910#tel d_g4911#d_g4912#d_g4913#d_g4914#d_g4915#tel d_g4916#d_g4917#4918#d_g4919#강남구 DINING 가게 4920개 중간저장
30264.282062768936 소요
d_g4920#d_g4921#d_g4922#d_g4923#d_g4924#d_g4925#d_g4926#d_g4927#tel d_g4928#d_g4929#d_g4930#d_g4931#d_g4932#d_g4933#d_g4934#d_g4935#d_g4936#d_g4937#d_g4938#d_g4939#d_g4940#d_g4941#d_g4942#tel d_g4943#d_g4944#d_g4945#d_g4946#tel d_g4947#4948#4949#강남구 DINING 가게 4950개 중간저장
30500.17535686493 소요
4950#d_g4951#4952#4953#d_g4954#d_g4955#4956#4957#d_g4958#d_g4959#d_g4960#d_g4961#tel d_g4962#d_g4963#d_g4964#d_g4965#d_g4966#d_g4967#d_g4968#tel d_g4969#d_g4970#d_g4971#d_g4972#d_g4973#d_g4974#d_g4975#d_g4976#d_g4977#d_g4978#d_g4979#강남구 DINING 가게 4980개 중간저장
30740.99929857254 소요
d_g4980#d_g4981#d_g4982#tel d_g4983#tel d_g4984#d_g4985#d_g4986#tel d_g4987#d_g4988#tel d_g4989#d_g4990#tel d_g4991#d_g4992#tel d_g4993#d_g4994#d_g4995#d_g4996#d_g4997#d_g4998#d_g4999#

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 94.0.4606
[WDM] - Get LATEST driver version for 94.0.4606
[WDM] - Driver [C:\Users\N\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


d_g5000#d_g5001#d_g5002#d_g5003#d_g5004#d_g5005#

In [ ]:
df = pd.read_csv('data/가게/가게정보_강남구_DINING.csv')

In [ ]:
df.shape